In [3]:
import json
from docx import Document
import re
import numpy as np

def calculate_heading_score(paragraph, previous_paragraph=None, next_paragraph=None):
    """
    Tính điểm tiêu đề với nhiều yếu tố kết hợp.
    """
    text = paragraph.text.strip()
    if not text:
        return 0

    score = 0
    style_name = paragraph.style.name.lower()
    word_count = len(text.split())

    # Điểm theo style
    if "heading 1" in style_name:
        score += 40
    elif "heading 2" in style_name:
        score += 30
    elif "heading 3" in style_name:
        score += 20

    # Độ dài và cấu trúc
    score += max(0, 15 - min(word_count, 15))
    if text.isupper():
        score += 20
    elif text.istitle():
        score += 10
    if text.endswith(":"):
        score += 10

    # Từ khóa tiêu đề
    heading_keywords = ["chương", "phần", "mục", "kết luận", "lời mở đầu", "bước"]
    for keyword in heading_keywords:
        if keyword in text.lower():
            score += 25
            break

    # Pattern số thứ tự
    if re.match(r"^\d+(\.\d+)*\s+[A-Z]", text) or re.match(r"^[IVXLCDM]+\.\s+[A-Z]", text):
        score += 20

    # Ngữ cảnh đoạn trước và sau
    if previous_paragraph and len(previous_paragraph.text.strip()) > 100 and word_count < 10:
        score += 10
    if next_paragraph and len(next_paragraph.text.strip()) > 200 and word_count < 10:
        score += 10

    return score

def adaptive_normalize(scores):
    """
    Chuẩn hóa điểm số theo phân phối.
    """
    scores = np.array(scores)
    if len(scores) == 0:
        return scores
    
    mean = np.mean(scores)
    std = np.std(scores)

    normalized_scores = (scores - mean) / (std + 1e-6)
    return normalized_scores * 20 + 50

def detect_heading_level(score):
    """
    Xác định cấp độ tiêu đề dựa trên điểm.
    """
    if score >= 90:
        return "heading 1"
    elif score >= 85:
        return "heading 2"
    elif score >= 80:
        return "heading 3"
    else:
        return "normal"

def greedy_select_headings(paragraphs, normalized_scores, min_distance=3):
    """
    Thuật toán tham lam để chọn heading đáng tin cậy nhất.
    """
    selected_headings = []
    last_selected_index = -min_distance

    sorted_indices = np.argsort(normalized_scores)[::-1]

    for idx in sorted_indices:
        if normalized_scores[idx] < 40:
            break

        # Đảm bảo khoảng cách giữa các heading
        if idx - last_selected_index >= min_distance:
            level = detect_heading_level(normalized_scores[idx])
            selected_headings.append({
                "index": idx,
                "text": paragraphs[idx].text.strip(),
                "level": level,
                "score": normalized_scores[idx]
            })
            last_selected_index = idx

    return sorted(selected_headings, key=lambda x: x["index"])

def process_docx(file_path, output_json_path):
    """
    Đọc file docx, phân tích heading và xuất ra file JSON.
    """
    doc = Document(file_path)
    paragraphs = doc.paragraphs
    n = len(paragraphs)

    scores = []
    results = []

    for i, para in enumerate(paragraphs):
        previous_para = paragraphs[i - 1] if i > 0 else None
        next_para = paragraphs[i + 1] if i < n - 1 else None
        score = calculate_heading_score(para, previous_para, next_para)
        scores.append(score)

    normalized_scores = adaptive_normalize(scores)
    selected_headings = greedy_select_headings(paragraphs, normalized_scores)

    # Đưa toàn bộ văn bản ra JSON
    for i, para in enumerate(paragraphs):
        result = {
            "index": i,
            "text": para.text.strip(),
            "level": detect_heading_level(normalized_scores[i]),
            "score": round(normalized_scores[i], 2)
        }
        results.append(result)

    # Ghi vào file JSON
    with open(output_json_path, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, indent=4, ensure_ascii=False)

    return results

# Ví dụ sử dụng
file_path = "D:/HAM.docx"
output_json_path = "D:/output.json"
output = process_docx(file_path, output_json_path)

print(f"Kết quả đã lưu vào {output_json_path}")


Kết quả đã lưu vào D:/output.json
